In [7]:
import os
import json
import glob
import numpy as np

# Settings
base_path = "F:/setup files/downloads/ResultLastly/"
output_json_path = os.path.join(base_path, "pareto_fronts_PFG-2F-RK.json")

# Use a wildcard pattern to find all relevant JSON files
file_pattern = os.path.join(base_path, "L*.json")
files = glob.glob(file_pattern)

def is_dominated(point, others):
    """
    Check if 'point' is dominated by any point in 'others'.
    A point p is dominated by q if q is no worse in all objectives and better in at least one.
    """
    return np.any(np.all(others <= point, axis=1) & np.any(others < point, axis=1))

def compute_pareto_front(solutions):
    """
    Compute the Pareto front from a set of solutions.
    Each solution is assumed to be a numpy array of objective values.
    """
    if len(solutions) == 0:
        return np.array([])
    
    solutions = np.array(solutions)
    pareto_mask = np.ones(solutions.shape[0], dtype=bool)
    
    for i, sol in enumerate(solutions):
        if pareto_mask[i]:
            # If any other solution dominates this one, mark as non-efficient
            pareto_mask[i] = not is_dominated(sol, solutions[np.arange(len(solutions)) != i])
            
            # Also remove any solutions dominated by this one
            for j in range(i+1, len(solutions)):
                if pareto_mask[j] and np.all(sol <= solutions[j]) and np.any(sol < solutions[j]):
                    pareto_mask[j] = False
    
    return solutions[pareto_mask]

pareto_fronts = {}

for file_path in files:
    file_name = os.path.basename(file_path)
    
    # Load JSON data from file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Gather all solutions at generation 100 across all algorithms
    all_solutions_gen100 = []
    for algorithm in ["PFG-EMOA"]:
        gen_key = "100"
        if gen_key in data[algorithm]:
            solutions = data[algorithm][gen_key]
            # Ensure solutions are numeric arrays
            all_solutions_gen100.extend(solutions)
    
    # Compute Pareto front from all gathered solutions
    pareto_front = compute_pareto_front(all_solutions_gen100)
    
    # Store the Pareto front (convert to list for JSON serialization)
    pareto_fronts[file_name] = pareto_front.tolist()


# Save all computed Pareto fronts to a JSON file
with open(output_json_path, 'w') as outfile:
    json.dump(pareto_fronts, outfile, indent=2)

print(f"Global Pareto fronts at generation 100 saved to {output_json_path}")

Global Pareto fronts at generation 100 saved to F:/setup files/downloads/ResultLastly/pareto_fronts_PFG-2F-RK.json


In [ ]:
import os
import json
import pandas as pd
import numpy as np

# -------------------- Configuration -------------------- #

# Define the base directory where your files are located
base_path = "F:/setup files/downloads/ResultLastly/"

# Path to the Pareto fronts JSON file
pareto_json_filename = "pareto_fronts_PFG-2F-RK.json"
pareto_json_path = os.path.join(base_path, pareto_json_filename)

# Path to the single-objective results CSV file
single_result_csv_filename = "single_objective_results.csv"  # Replace with your actual CSV filename
single_result_csv_path = os.path.join(base_path, single_result_csv_filename)

# Path to save the output CSV file
output_csv_filename = "pareto_vs_single_domination_counts.csv"
output_csv_path = os.path.join(base_path, output_csv_filename)

# -------------------- Load Pareto Fronts -------------------- #

# Load Pareto fronts from JSON
with open(pareto_json_path, 'r') as f:
    pareto_fronts = json.load(f)

print(f"Loaded Pareto fronts from {pareto_json_path}")

# -------------------- Load Single-Objective Results -------------------- #

# Load single-objective results from CSV
single_results = pd.read_csv(single_result_csv_path)

print(f"Loaded single-objective results from {single_result_csv_path}")

# Verify the required columns exist
required_columns = ['Instance', 'EC', 'VF', 'CF', 'MWT']  # Adjust 'CF1', 'CF2' as per your CSV
for col in required_columns:
    if col not in single_results.columns:
        raise ValueError(f"Missing required column '{col}' in the CSV file.")

# -------------------- Define Dominance Function -------------------- #

def dominates(pareto_solution, single_solution):
    """
    Check if a Pareto solution dominates the single solution.
    Dominance is defined as being larger or equal in all objectives.
    
    Parameters:
    - pareto_solution: numpy array of Pareto solution objectives
    - single_solution: numpy array of single solution objectives
    
    Returns:
    - Boolean indicating whether pareto_solution dominates single_solution
    """
    return np.all(pareto_solution >= single_solution)

# -------------------- Compare and Count Dominations -------------------- #

# Initialize a list to store domination counts
domination_counts = []

# Iterate over each row in the single-objective results
for idx, row in single_results.iterrows():
    instance = row['Instance']
    
    # Extract single solution objectives
    # Adjust the order and names based on your specific objectives
    single_solution = np.array([row['EC'], row['CF1'], row['CF2'], row['MWT']])
    
    # Retrieve the corresponding Pareto front
    pareto_front = pareto_fronts.get(instance, [])
    
    if not pareto_front:
        print(f"Warning: No Pareto front found for instance '{instance}'.")
        domination_count = 0
    else:
        # Convert Pareto front to numpy array for vectorized operations
        pareto_front_array = np.array(pareto_front)
        
        # Check dominance for each Pareto solution
        # Assuming all objectives are to be maximized
        dominance = np.all(pareto_front_array >= single_solution, axis=1)
        
        # Count the number of dominating solutions
        domination_count = np.sum(dominance)
    
    # Append the result
    domination_counts.append({
        'Instance': instance,
        'Dominating_Solutions_Count': int(domination_count)
    })

# -------------------- Save Results -------------------- #

# Create a DataFrame from the results
domination_df = pd.DataFrame(domination_counts)

# Save the domination counts to a CSV file
domination_df.to_csv(output_csv_path, index=False)

print(f"Domination counts saved to {output_csv_path}")